<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%86%92%E9%9A%AA24_%E3%80%8A%E7%B4%85%E6%A8%93%E5%A4%A2%E3%80%8B%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入套件

In [2]:
import tensorflow as tf
import pickle

from tensorflow.keras.models import load_model, model_from_json

### 2. 讀入訓練好的 RNN 紅樓夢生成器

In [3]:
!wget --no-check-certificate \
    https://github.com/yenlung/Python-AI-Book/blob/main/dream_rnn.zip?raw=true \
    -O /content/dream_rnn.zip

--2023-02-19 09:52:39--  https://github.com/yenlung/Python-AI-Book/blob/main/dream_rnn.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/yenlung/Python-AI-Book/raw/main/dream_rnn.zip [following]
--2023-02-19 09:52:39--  https://github.com/yenlung/Python-AI-Book/raw/main/dream_rnn.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream_rnn.zip [following]
--2023-02-19 09:52:39--  https://raw.githubusercontent.com/yenlung/Python-AI-Book/main/dream_rnn.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting resp

In [4]:
import os
import zipfile

local_zip = '/content/dream_rnn.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
f = open('architecture.json', 'r')
loaded_model = f.read()
f.close()

In [6]:
model = model_from_json(loaded_model)

In [7]:
model.load_weights("weights.h5")

【更新】TensorFlow 更新後, 我們發現需要這個步驟。

In [10]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 1.7 MB/s eta 0:00:00


In [11]:
f = open('tokenizer.pkl', 'rb')
tokenizer = pickle.load(f)
f.close()

#### 【已下載紅樓夢生成器請執行這段】

如果你已經在[我的 GitHub](https://github.com/yenlung/Deep-Learning-Basics) 中下載紅樓夢生成模型, 這包括:

1. `dream_rnn` 資料夾, 這是模型和訓練好的權重。
2. `dream_tokenizer2.pkl` 檔案, 這是模型使用的 tokenizer。

存到你的 Google Drive, 放 Colab 程式的地方 (預設是 `Colab Notebooks` 資料夾)。那執行下面這一段, 不用再讀入一次模型資料。

In [ ]:
#from google.colab import drive

#drive.mount('/content/drive')

In [ ]:
#%cd '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
#model = load_model('dream_rnn')

In [ ]:
#f = open('dream_tokenizer2.pkl', 'rb')
#tokenizer = pickle.load(f)
#f.close()

### 3. 打造紅樓夢生成器

In [12]:
max_id = len(tokenizer.word_index)

In [13]:
max_id

4522

接下來是一段文字, 我們用事先訓練好的 tokenizer 換成一段數字, 最後用 one-hot encoding 回傳。

In [14]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences([texts]))-1
    return tf.one_hot(X, max_id)

這段程式主要依輸入的一段文字, 用我們的 model 去預測下一個字。注意像平常的分類問題, 這裡輸出是每人個字出現機率最高的。但都照這樣, 我們輸入同一段文字, 之後出現的文字永遠是一樣的! 常用的手法是去設定 `temperature`, `temperature` 接近 0, 大致上就取機率最高的字; `temperature` 越大就越隨機。太隨機就變成亂數取字! 一般 `temperature`設 1 左右效果最佳。

In [15]:
def next_char(texts, temperature=1):
    X_new = preprocess(texts)
    y_predict = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_predict) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

最後就一段文字進來，再產生 `n_chars` 這麼多個字。我們原本一段文字只能生一個字, 那就一次生一個字, 最後要生多少個字就生多少個字。

原本訓練我們一段是 100 個字去訓練的, 這裡超過 100 字時我們就取最後 100 個字丟入模型。

In [16]:
def complete_text(texts, n_chars=50, temperature=1):
    n_chars=int(n_chars)
    for _ in range(n_chars):
        texts = texts + next_char(texts[-100:], temperature)
    return texts

### 4. 測試紅樓夢生成器

In [19]:
complete_text("自孫悟空從石頭中蹦出來之後，", n_chars=300, temperature=1.2)

1/1 [==============================] - 0s 48ms/step


'自孫悟空從石頭中蹦出來之後，先飛報單的。賈政嘆口氣。手又賞了一本，與他老先邀雨村回話。\n\n那瓦礫明乍寒了一番補入，毫不進，說著：「悟如從此，從天入三洞天。因開燈，古道今日，原來如今的奇處，顏色大大道如此。」士隱自趕的道：「二老爺通報仙草已經失了那仙翁，你把這條猩紅氈的斗篷，向小子同賈珍珠方相認，俟薛姨媽那邊設停寺內。\n\n次日，鳳姐等分頭取藥那劉姥姥家的，眾人打發人去給邢夫人辭了眾人。王夫人命賈蘭跟了侍兒去，分頭只一顧頭回來看著賈環的相思。王夫人便告訴了平兒，在賈芸家外沒有，不敢答言，只有好心盛妝。\n\n那空空牢牢記著此言，又把五更五更收心起題，寫著書去。回奏：「皇上隆恩，內眷們不及大理的。」賈政帶著賈蘭場中迷失。\n\n賈政回'

### 5. 用 `gradio` 做成一個網路 app

我們準備用 [`gradio`](https://gradio.app/) 套件, 神速做完一個 web app。最酷的是, 最後出現 `https://xxxx.gradio.app` 那個網址, 在你的 Colab 還在執行的時候, 任何人都可以用任何瀏覽器連進來使用!

In [20]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [21]:
import gradio as gr

In [22]:
iface = gr.Interface(
    fn=complete_text,
    inputs=[
        "text",
        gr.Slider(minimum=50, maximum=200, value=50, step=1),
        gr.Slider(minimum=0.2, maximum=2, value=1, step=0.2)],
    outputs="text",
    title="紅樓夢生成器",
    description="起個頭, 幫你完成一段紅樓夢。可以改變 temperature, 越小生出的字越固定, 越大越隨機。")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1f503da7-20ec-4fbe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
